<a href="https://colab.research.google.com/github/lianghunan17/python/blob/master/EXCEL_%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E2%97%8B%E4%BB%98%E3%81%8D%E3%83%81%E3%82%A7%E3%83%83%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openpyxl
from openpyxl.styles import PatternFill, Alignment
from openpyxl.utils import get_column_letter

# Excel ファイルを開く
wb = openpyxl.load_workbook('/content/データ仕分けツール新 .xlsx')

# "データ_仕分け" シートを取得
data_sheet = wb['データ_仕分け']

# B8 セルから始まるB列の全てのシート名を取得
sheet_names = [cell[0].value for cell in data_sheet['B8':f'B{data_sheet.max_row}'] if cell[0].value is not None]

# 新しいWorkbookを作成
new_wb = openpyxl.Workbook()

# 最初のSheetを削除
default_sheet = new_wb['Sheet']
new_wb.remove(default_sheet)

# 各シート名に対して処理を実行
for sheet_name in sheet_names:
    sheet = wb[sheet_name.strip()]  # シート名の前後の空白を削除

    # D列のユニークな値を取得
    unique_D_values = list(set([cell.value for cell in sheet['D'] if cell.value is not None]))

    # C列のユニークな値を取得
    unique_C_values = list(set([cell.value for cell in sheet['C'] if cell.value is not None]))

    # 新しいシートを作成
    new_sheet = new_wb.create_sheet(title=f"{sheet_name.strip()}〇")

    # 新しいシートにD列のユニークな値をB2から横並びに張り付ける
    for i, value in enumerate(unique_D_values, start=2):
        new_sheet.cell(row=2, column=i, value=value)

    # 新しいシートにC列のユニークな値を張り付ける
    for i, unique_C_value in enumerate(unique_C_values, start=3):
        new_sheet.cell(row=i, column=1, value=unique_C_value)

    # 黄色のパターンフィルを作成
    yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

    # データ_仕分けシートでB2から始まる行値とA列値を参照して検索し、該当する場合は新しいシートに〇をつける
    for row in sheet.iter_rows(min_row=2, min_col=1, max_col=sheet.max_column, values_only=True):
        for i, unique_D_value in enumerate(unique_D_values, start=2):
            for j, unique_C_value in enumerate(unique_C_values, start=3):
                if row[3] == unique_D_value and row[2] == unique_C_value:  # Excelの列は0からではなく1から始まるので注意
                    new_sheet.cell(row=j, column=i, value="〇")
                    new_sheet.cell(row=j, column=i).fill = yellow_fill

    # セルの幅を自動調整 (2行目以降の列のみ)
    for column in new_sheet.iter_cols(min_col=1, min_row=1):
        max_length = max(len(str(cell.value)) for cell in column)
        column_letter = get_column_letter(column[0].column)
        new_sheet.column_dimensions[column_letter].width = max_length

    # セルrange("A2")にデータを追加
    new_sheet['A2'].value = 'ー'
   
    # 新しいシートの行2と3にフィルターを付ける
    new_sheet.auto_filter.ref = f"A2:{get_column_letter(new_sheet.max_column)}3"


    # データをすべて中央に揃える
    for row in new_sheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal="center")

    # ウインドウ枠の固定 (B2)
    new_sheet.freeze_panes = new_sheet['B2']

    


    # 新しいWorkbookを保存
    new_wb.save('/content/○付き確認.xlsx')
